Importing libraries

In [ ]:
import os #to play with the directories
import tensorflow as tf
import matplotlib.pyplot as plt

Dataset normalization

In [1]:
def preprocess_img(image,label,image_size=224):
  return tf.cast(image/255. ,tf.float32),label

Batching our Dataset

In [ ]:
train_data = train_data.map(map_func=preprocess_img,num_parallel_calls=tf.data.AUTOTUNE) # use preprocess_img function for all of the train_data, and autotune is used to tell gpu to use all of the computational power
train_data=train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

test_data = test_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)

test_data = test_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE).cache()

Feature Extraction Model without fine tuning(all layers are frezed)

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

inputs = layers.Input(shape=input_shape, name="input_layer", dtype=tf.float16)
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D(name="pooling_layer")(x)
x = layers.Dense(len(labels))(x) 
outputs = layers.Activation("softmax", dtype=tf.float32, name="softmax_float32")(x) 
model = tf.keras.Model(inputs, outputs)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

Fiting the training data on the above feature extraction model.

In [ ]:
accuracy_on_model_with_freezed_layer = model.fit(train_data, epochs=3,steps_per_epoch=len(train_data),validation_data=test_data,validation_steps=int(0.15 * len(test_data)),)

Evaluating our Model on the feature extraction model without fine tuning


In [ ]:
model.evaluate(test_data)

Unfreezing layers of the EfficientNetB0

In [ ]:
for layer in model.layers:
  layer.trainable=True

Now compiling the model with fine tuning i.e. unfreezed layer

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(lr=0.0001),
              metrics=["accuracy"])

Evaluating our Model on the feature extraction model with fine tuning

In [ ]:
accuracy_from_fine_tuned_model = model.fit(train_data, epochs=100,steps_per_epoch=len(train_data),validation_data=test_data,validation_steps=int(0.15 * len(test_data)),)